In [1]:
import json
import logging
import os
import random
import re
import subprocess
from pathlib import Path
from typing import Literal

import jinja2
import openai
from dotenv import load_dotenv
from faker import Faker
from openai import Client
from pydantic import BaseModel, Field


load_dotenv()

llm_api_url = os.getenv("LLM_API_URL", default="http://localhost:11434/v1")
llm_api_key = os.getenv("LLM_API_TOKEN", default="ollama")
llm_model = os.getenv("LLM_API_MODEL", default="qwen2.5-coder:7b")

client = Client(base_url=llm_api_url, api_key=llm_api_key)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("logs/resume_generation.log", encoding="utf-8"), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)
openai._utils._logs.logger.setLevel(logging.WARNING)  # noqa: SLF001
openai._utils._logs.httpx_logger.setLevel(logging.WARNING)  # noqa: SLF001

In [2]:
client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "test",
        }
    ],
    model=llm_model,
).choices[0].message.content

'Hello! It seems like you might have sent a test message. How can I assist you today? If you have any questions or need help with anything specific, feel free to let me know!'

In [8]:
class Experience(BaseModel):
    job_title: str = Field(..., description="Должность")
    company: str = Field(..., description="Компания")
    start_date: str = Field(..., description="Дата начала работы")
    end_date: str | None = Field(None, description="Дата окончания работы")
    achievements: list[str] | None = Field(None, description="Достижения расписанные по методике STAR")


class Education(BaseModel):
    degree: str = Field(..., description="Степень (например, бакалавр, магистр и т.д.)")
    institution: str = Field(..., description="Учебное заведение")
    start_date: str = Field(..., description="Дата начала учебы")
    end_date: str | None = Field(None, description="Дата окончания учебы")
    details: str = Field(..., description="Cпециализация, (например: Машинное обучение и высоконагруженные системы)")


class Projects(BaseModel):
    name: str = Field(..., description="Название проекта, например парсер резюме")
    link: str = Field(..., description="Ссылка на проект, например на github")
    description: str = Field(..., description="Описание проекта")


class Contacts(BaseModel):
    phone: str = Field(..., description="Мобильный телефон")
    email: str = Field(..., description="Email")
    linkedin: str | None = Field(None, description="LinkedIn")
    github: str | None = Field(None, description="Ссылка на Github, только если кандидат из IT сферы!")
    location: str = Field(..., description="Текущая локация кандидата")


class Resume(BaseModel):
    name: str = Field(..., description="Полное имя")
    contact_info: Contacts = Field(..., description="Контактная информация")
    gender: Literal["мужской", "женский"] = Field(..., description="Пол кандидата")
    title: str = Field(..., description="Желаемая должность")
    summary: str = Field(..., description="Краткое summary о себе")
    skills: list[str] | None = Field(None, description="Перечень навыков")
    experience: list[Experience] | None = Field(None, description="Опыт работы, придумай в соответствии с years_of_experience")
    education: list[Education] = Field(None, description="Образование")
    languages: list[str] | None = Field(None, description="Языки, на которых говорит кандидат")
    certifications: list[str] | None = Field(None, description="Сертификаты (если есть)")
    hobbies: list[str] | None = Field(None, description="Хобби, опционально, например: бокс")
    portfolio: list[Projects] | None = Field(None, description="Опционально, портфолио и примеры работ")


In [11]:
fake = Faker(locale="ru_RU")

LATEX_ESCAPE_MAP = {
    "%": r"\%",
    "$": r"\$",
    "&": r"\&",
    "#": r"\#",
    "_": r"\_",
    "{": r"\{",
    "}": r"\}",
    "~": r"\textasciitilde{}",
    "^": r"\textasciicircum{}",
}

jobs = [
    "Data Engineer",
    "Developer",
    "Business Expert",
    "System Analyst",
    "Data Scientist",
    "Support Engineer",
    "Technical Lead",
    "Agile Expert",
    "UX/UI Designer",
    "Tester",
    "Product Owner",
    "Software Engineer",
    "Data Analyst",
    "Researcher",
    "Software Architect",
    "Process Lead",
    "Data Steward",
    "Специалист по охране труда",
    "Product Analyst",
    "Контент-менеджер",
    "Руководитель",
    "Кредитный эксперт",
    "специалист по расчету заработной платы",
    "Ассистент",
    "Кассир-контролер",
    "Графический дизайнер",
    "Продуктовый редактор",
    "Юрист",
    "Менеджер проектов",
    "Менеджер по продажам",
    "Менеджер поддержки клиентов",
    "Менеджер по развитию ипотечного бизнеса",
]

prompt = """
    Ты кандидат с такими вводными:
    {candidate}
    Ты ищешь работу и тебе нужно сгенировать продающее резюме.
    Задача:
    1. Придумай и напиши краткое описание профессионального опыта и квалификации кандидата, используя 2-3 предложения. Учитывай сферу деятельности кандидата (не обязательно IT).
    2. Придумай список из 5-10 ключевых навыков, которые могут быть полезны для указанной должности, в том числе для сфер, не связанных с IT.
    3. Опиши опыт работы кандидата. Укажи места работы с должностью, компанией и опиши достижения кандидата, используя метод STAR (Situation, Task, Action, Result), но в соответсвтии с years_of_experience.
    4. Укажи образование кандидата, включая степень (например, бакалавр, магистр) и учебное заведение. Если есть, добавь дополнительные сведения, такие как специализация.
    5. Опиши языки, которыми владеет кандидат (если применимо).
    6. Опционально, придумай 1-2 сертификата, связанные с профессиональной областью кандидата (включая, например, управление проектами, финансы, маркетинг и т.д.).
    7. Опционально, придумай 1-3 хобби кандидата, которые отражают его интересы и могут быть связаны с профессиональной деятельностью.
    8. Опционально, предоставь информацию о реализованных проектах, а также конкретных результатах, достижениях и внедрениях.
    """

def escape_string(text: str | None) -> str | None:
    if not text:
        return text
    text = re.sub(r"[\"']", "", text)
    for char, replacement in LATEX_ESCAPE_MAP.items():
        text = text.replace(char, replacement)
    return text


def escape_all_strings(obj: any) -> any:
    if isinstance(obj, str):
        return escape_string(obj)
    if isinstance(obj, list):
        return [escape_all_strings(item) for item in obj]
    if isinstance(obj, dict):
        return {k: escape_all_strings(v) for k, v in obj.items()}
    if isinstance(obj, BaseModel):
        return escape_all_strings(obj.model_dump())
    return obj


def clean_cv_folder(cv_dir: Path) -> None:
    for file in cv_dir.iterdir():
        if file.suffix != ".pdf":
            file.unlink()


def generate_random_resume() -> dict:
    return {
        "name": fake.name(),
        "phone_number": fake.phone_number(),
        "city": fake.city(),
        "desired_job": random.choice(jobs),  # noqa: S311
        "years_of_experience": random.randint(0, 15),  # noqa: S311
    }

jinja_env = jinja2.Environment(
    loader=jinja2.FileSystemLoader("src/cv_generator/config"),
    comment_start_string="<%",
    comment_end_string="%>",
    autoescape=True,
)
template = jinja_env.get_template("cv_template.tex")


def save_resume_to_file(resume: Resume, folder: str = "data/resumes_json") -> Path:
    Path(folder).mkdir(parents=True, exist_ok=True)
    filename = f"{resume.name.lower().replace(' ', '_')}.json"
    path = Path(folder) / filename
    with path.open("w", encoding="utf-8") as f:
        json.dump(resume.model_dump(), f, ensure_ascii=False, indent=4)
    return path

In [12]:
completion = client.beta.chat.completions.parse(
    model=llm_model,
    messages=[
        {"role": "system", "content": "Твоя задача создать структурированный вывод согласно заданной схеме."},
        {"role": "user", "content": prompt.format(candidate=generate_random_resume())},
    ],
    temperature=0.15,
    response_format=Resume,
)
answer = completion.choices[0].message.parsed


In [ ]:
json_file_path = save_resume_to_file(answer)
logger.info(f"Json is saved: {json_file_path}")

resume = escape_all_strings(answer)
candidate_name = "_".join(resume["name"].split()).lower()
latex_file_path = latex_folder / f"cv_{candidate_name}.tex"
output = template.render(resume)
latex_file_path.write_text(output, encoding="utf-8")
logger.info(f"LaTeX is saved: {latex_file_path}")


In [5]:
cv_folder = Path("data/cv")
cv_folder.mkdir(parents=True, exist_ok=True)
latex_folder = Path("data/latex")
latex_folder.mkdir(parents=True, exist_ok=True)
fake = Faker(locale="ru_RU")

N = 50
for i in range(N):
    try:
        logger.info(f"Generating resume {i + 1}/{N}")

        completion = client.beta.chat.completions.parse(
            model=llm_model,
            messages=[
                {"role": "system", "content": "Твоя задача создать структурированный вывод согласно заданной схеме."},
                {"role": "user", "content": prompt.format(candidate=generate_random_resume())},
            ],
            temperature=0.15,
            response_format=Resume,
        )
        answer = completion.choices[0].message.parsed

        json_file_path = save_resume_to_file(answer)
        logger.info(f"Json is saved: {json_file_path}")

        resume = escape_all_strings(answer)
        candidate_name = "_".join(resume["name"].split()).lower()
        latex_file_path = latex_folder / f"cv_{candidate_name}.tex"
        output = template.render(resume)
        latex_file_path.write_text(output, encoding="utf-8")
        logger.info(f"LaTeX is saved: {latex_file_path}")

        subprocess.run(
            [
                "/opt/homebrew/bin/pdflatex",
                "-interaction=nonstopmode",
                f"-output-directory={cv_folder}",
                str(latex_file_path),
            ],
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
        logger.info(f"pdf is compilated: {latex_file_path.with_suffix('.pdf')}")

    except subprocess.CalledProcessError:
        logger.exception(f"latex compilation error: {latex_file_path.name}")
    except Exception:
        logger.exception(f"generating resume error {i}")

clean_cv_folder(cv_folder)
logger.info("temp files cleaned")

2025-04-08 23:55:28,619 [INFO] Generating resume 1/50
2025-04-08 23:56:12,158 [INFO] Json is saved: data/resumes_json/казаков_ефим_арсеньевич.json
2025-04-08 23:56:12,163 [INFO] LaTeX is saved: data/latex/cv_казаков_ефим_арсеньевич.tex
2025-04-08 23:56:12,853 [INFO] pdf is compilated: data/latex/cv_казаков_ефим_арсеньевич.pdf
2025-04-08 23:56:12,854 [INFO] Generating resume 2/50
2025-04-08 23:56:55,548 [INFO] Json is saved: data/resumes_json/федотов_спиридон_антипович.json
2025-04-08 23:56:55,553 [INFO] LaTeX is saved: data/latex/cv_федотов_спиридон_антипович.tex
2025-04-08 23:56:56,134 [INFO] pdf is compilated: data/latex/cv_федотов_спиридон_антипович.pdf
2025-04-08 23:56:56,135 [INFO] Generating resume 3/50
2025-04-08 23:57:50,545 [INFO] Json is saved: data/resumes_json/герман_бориславович_антонов.json
2025-04-08 23:57:50,554 [INFO] LaTeX is saved: data/latex/cv_герман_бориславович_антонов.tex
2025-04-08 23:57:51,201 [INFO] pdf is compilated: data/latex/cv_герман_бориславович_антонов

In [5]:
DATA_DIR = Path("data")
CV_DIR = DATA_DIR / "resumes_pdf"
LATEX_DIR = DATA_DIR / "resumes_latex"
JSON_DIR = DATA_DIR / "resumes_json"


def clean_cv_folder(cv_dir: Path = CV_DIR) -> None:
    for file in cv_dir.iterdir():
        if file.suffix != ".pdf":
            file.unlink()

clean_cv_folder()

In [17]:
json_file_path = "json/test.json"


latex_file_path = LATEX_DIR / f"{Path(json_file_path).stem}.tex"
(CV_DIR / latex_file_path.name).with_suffix(".pdf")

PosixPath('data/resumes_pdf/test.pdf')

In [12]:
from pathlib import Path

from cv_generator.config.config import settings


Path(settings.latex_template_path).name

PosixPath('cv_generator/config/cv_template.tex')